### Import Libraries

In [1]:
from hyperas.distributions import uniform

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras import backend
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# Versioning
print(keras.__version__)
print(tf.__version__)

2.2.4
1.10.0


### Read in Normalized Option Pricing Data Set

In [4]:
option_df = pd.read_pickle("opt_data_z.pkl")
## Move data into numpy array
prices = option_df.values

In [5]:
# X values
X = prices[:,:4]

# Y values 
y = prices[:,4:5]

# Use Sklearn library to randomly split data into training/testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Build Keras/Scikit Learn Function

In [6]:
def create_model():

    nodes = 200

    # Create Model
    model = Sequential()

    # Layer 1
    model.add(Dense(nodes, input_shape=(4,))) 
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    # Layer 2
    model.add(Dense(nodes)) 
    model.add(Activation ('relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    # Layer 3
    model.add(Dense(nodes))
    model.add(Activation ('relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    # Layer 4 - Output Layer
    model.add(Dense(1))
    model.add(Activation('elu'))
    
    model.compile(loss='mse', optimizer = 'rmsprop', metrics=['accuracy'])
    
    result = model.fit(X_train, y_train,
              batch_size=64,
              epochs=2,
              verbose=2,
              validation_split=0.1)
    
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [7]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [16]:
from __future__ import print_function
import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform


def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    nb_classes = 10
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # If we choose 'four', add an additional fourth layer
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(100))

        # We can also choose between complete sets of layers

        model.add({{choice([Dropout(0.5), Activation('linear')])}})
        model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    result = model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              epochs=2,
              verbose=2,
              validation_split=0.1)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}




In [17]:
if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials())
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

OSError: [Errno 22] Invalid argument: 'C:\\Users\\blues\\Desktop\\Python\\blackscholes_nas\\<ipython-input-17-b978c4c90872>'

In [14]:
data = X_train, y_train, X_test, y_test 
best_run, best_model = optim.minimize(model=create_model,
                                          data= data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials())


OSError: [Errno 22] Invalid argument: 'C:\\Users\\blues\\Desktop\\Python\\blackscholes_nas\\<ipython-input-14-8073bdcfa1c7>'

#### Instantiate Keras Classifier

In [34]:
neural_network = KerasClassifier(build_fn = create_model, verbose = 0)

#### Define Search Parameters 

In [36]:
epochs = [5, 10]
batches = [5, 10]
optimizers = ['rmsprop', 'adam']

hyperparameters = dict(optimizer=optimizers, epochs=epochs)

In [39]:
callbacks_list = [EarlyStopping(monitor = 'loss', patience = 1)]

In [ ]:
# Create grid search
grid = GridSearchCV(estimator = neural_network, param_grid = hyperparameters, cv = 5)

# Fit grid search
grid_result = grid.fit(X_train, y_train, verbose = 2, callbacks= callbacks_list)

Epoch 1/5
 - 54s - loss: 278407250.5027 - acc: 3.8473e-05
Epoch 2/5
 - 52s - loss: 111185377.3529 - acc: 3.8473e-05
Epoch 3/5
 - 52s - loss: 95286021.6036 - acc: 3.4626e-05
Epoch 4/5
 - 51s - loss: 84456209.2750 - acc: 6.9252e-05
Epoch 5/5
 - 52s - loss: 71911055.9303 - acc: 5.7710e-05
Epoch 1/5
 - 52s - loss: 314273175.6582 - acc: 2.6931e-05
Epoch 2/5
 - 52s - loss: 114244877.4193 - acc: 5.3863e-05
Epoch 3/5
 - 52s - loss: 95644518.3819 - acc: 4.8092e-05
Epoch 4/5
 - 55s - loss: 86969550.6744 - acc: 1.9237e-05
Epoch 5/5
 - 53s - loss: 78264517.7778 - acc: 4.8092e-05
Epoch 1/5
 - 150s - loss: 350528780.6335 - acc: 2.1160e-05
Epoch 2/5
 - 145s - loss: 151824017.4104 - acc: 2.5008e-05
Epoch 3/5
 - 145s - loss: 97588715.2068 - acc: 4.2321e-05
Epoch 4/5
 - 145s - loss: 88696925.0118 - acc: 4.4244e-05
Epoch 5/5
 - 145s - loss: 81593974.0458 - acc: 5.1939e-05
Epoch 1/5
 - 143s - loss: 350501651.1552 - acc: 2.8855e-05
Epoch 2/5
 - 146s - loss: 173938061.9479 - acc: 3.6550e-05
Epoch 3/5
 - 146

In [ ]:
grid_result.best_params_

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=10, validation_split=0.1, verbose = 2, callbacks= callbacks_list)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(y_test, predictions)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,1], [0,1], 'r')
plt.grid(True)

